In [2]:
import random

In [11]:
# Class imported from notebook 7 

class Hangman:
    def __init__(self):
        import requests
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code==200:
            self.content = str(r.content.decode()).split('\n')
        else:
            print("Erro: ", r.status_code)
    
    def new_game(self, lifes=5):
        self.lifes = lifes
        self.word = random.choice(self.content)
        return len(self.word)

    def guess_letter(self, letter):
        if self.lifes > 0:
            if letter in self.word:
                return [idx for idx in range(len(self.word)) if self.word[idx]==letter]
            else:
                self.lifes -= 1
                if self.lifes == 0:
                    print("You lost! Word: " + str(self.word))
                    return False
                else:
                    return []
        
    def guess_word(self, word):
        if self.lifes > 0:
            if self.word == word:
                print ("Win!")
                return True
            else:
                self.lifes = 0
                print("You lost! Word: " + str(self.word) )
                return False

In [18]:
# Showcase

game = Hangman()
letters = game.new_game()
print("Letters:" + str(letters))

game.guess_letter('2')
game.guess_letter('q')
game.guess_letter('a')
game.guess_letter('y')
game.guess_letter('z')
game.guess_letter('x')

'Lifes: ', game.lifes


Letters:13
You lost! Word: reembarcareis


('Lifes: ', 0)